左右または上下を反転させる行列Jの作用をベクトルからベクトルへの関数として実装

In [1]:
function J(y)
    n=length(y)
    Jy = Vector{eltype(y)}(n)
    for i=1:n
        Jy[i] = y[n-i+1]
    end
    return Jy
end

J (generic function with 1 method)

行列積は行列になるが、双対なベクトルをかけるとスカラーになる

In [2]:
[2 3] * [3 4]'

1×1 Array{Int64,2}:
 18

In [3]:
[2, 3]' * [3,4]

18

In [4]:
typeof([1]'*[1])

Int64

In [5]:
function Durbin(T)
    t = eltype(T)
    n = length(T)-1
    y0 = [T[2]/T[1]]
    for k = 1:(n-1)
        y1 = zeros(k+1)
        y1[k+1] = (T[k+2] - T[2:k+1]' * J(y0))/ (T[1] - T[2:k+1]' * y0)
        y1[1:k] = y0 - y1[k+1] * J(y0)
        y0 = y1
    end
    return y0
end

Durbin (generic function with 1 method)

In [22]:
function Levinson(T,b)
    n = length(T)-1
    r = T[2:n+1]
    u0 = [b[1]/T[1]]
    for k=1:(n-1)
        y = Durbin(T[1:k+1])
        u1 = zeros(k+1)
        u1[k+1] = (b[k+1] - r[1:k]' * J(u0)) / (T[1] - r[1:k]' * y)
        u1[1:k] = u0 - u1[k+1] * J(y)
        u0 = u1
    end
    return u0
end

Levinson (generic function with 1 method)

In [ ]:
function Trench(T)
    n = length(T)-1
    Tinv = Array{float64}(n,n)
    y = Durbin(T)
    r = T[2:n+1]
    μ = 1/(τ-r[1:k']*y)
    v = -μ*J(y)
    Tinv[1:n-1,n] = v
    Tinv[n,1:n-1] = v
    Tinv[n,n] = v
    Tinv[:,1] = J(Tinv[:,n])
    Tinv[1,:] = J(Tinv[n,:])
    

In [6]:
function T2Ab(T)
    n = length(T)-1
    A = Array{eltype(T)}(n,n)
    for i = 1:n
        for j = 1:n
            A[i,j] = T[abs(i-j)+1]
        end
    end
    b = T[2:n+1]
    return A,b
end

T2Ab (generic function with 1 method)

In [24]:
T = [1,2,3,4,5]
A,b = T2Ab(T);

In [8]:
A

4×4 Array{Int64,2}:
 1  2  3  4
 2  1  2  3
 3  2  1  2
 4  3  2  1

In [9]:
b

4-element Array{Int64,1}:
 2
 3
 4
 5

In [10]:
using BenchmarkTools

In [11]:
Durbin(T) #Durbinの解法

4-element Array{Float64,1}:
  1.2        
 -8.88178e-17
 -4.44089e-16
  0.2        

In [12]:
A \ b #built-in

4-element Array{Float64,1}:
  1.2        
  5.55112e-17
 -4.16334e-17
  0.2        

In [13]:
T = rand(10000)
A,b = T2Ab(T)

([0.198231 0.994168 … 0.788336 0.303988; 0.994168 0.198231 … 0.556599 0.788336; … ; 0.788336 0.556599 … 0.198231 0.994168; 0.303988 0.788336 … 0.994168 0.198231], [0.994168, 0.219243, 0.416845, 0.777861, 0.263206, 0.622764, 0.871123, 0.574871, 0.170267, 0.366087  …  0.784387, 0.0475305, 0.570837, 0.374071, 0.865201, 0.163185, 0.556599, 0.788336, 0.303988, 0.263036])

In [14]:
@benchmark Durbin(T)

BenchmarkTools.Trial: 
  memory estimate:  2.61 GiB
  allocs estimate:  125639
  --------------
  minimum time:     634.234 ms (14.09% GC)
  median time:      644.666 ms (14.18% GC)
  mean time:        649.810 ms (14.11% GC)
  maximum time:     674.702 ms (14.02% GC)
  --------------
  samples:          8
  evals/sample:     1

In [15]:
@benchmark A \ b

BenchmarkTools.Trial: 
  memory estimate:  762.94 MiB
  allocs estimate:  10
  --------------
  minimum time:     17.782 s (0.07% GC)
  median time:      17.782 s (0.07% GC)
  mean time:        17.782 s (0.07% GC)
  maximum time:     17.782 s (0.07% GC)
  --------------
  samples:          1
  evals/sample:     1

In [35]:
T = rand(10000)
A,_ = T2Ab(T)
b = rand(9999)

9999-element Array{Float64,1}:
 0.125509 
 0.414344 
 0.468819 
 0.920057 
 0.630179 
 0.844616 
 0.0923435
 0.990935 
 0.617321 
 0.971947 
 0.92809  
 0.648107 
 0.75163  
 ⋮        
 0.155405 
 0.674783 
 0.885988 
 0.930114 
 0.653436 
 0.417376 
 0.780981 
 0.336823 
 0.0990772
 0.0696222
 0.96354  
 0.721865 